In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import gymnasium as gym

from gymnasium.wrappers.time_limit import TimeLimit

from dice_rl_TU_Vienna.wrappers import AbsorbingWrapper
from dice_rl_TU_Vienna.plugins.stable_baselines3.policy import get_model_PPO
from dice_rl_TU_Vienna.dataset import get_dataset

from plugins.gymnasium.cartpole.config import *

/Users/richardweiss/Documents/.venvs/dice_rl_TU_Vienna/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [3]:
env = gym.make('CartPole-v0')
env_absorbing = AbsorbingWrapper(env, absorbing_rew=-1.0)
env_absorbing_time_limit = TimeLimit(env_absorbing, max_episode_steps)

/Users/richardweiss/Documents/.venvs/dice_rl_TU_Vienna/lib/python3.11/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [4]:
policy = {}
id_policy = {}

for kind in kinds:
    x, y = get_model_PPO(
        dir_base,
        env,
        {
            "seed": seed,
            "total_timesteps": total_timesteps[kind],
        },
        verbosity=1,
    )

    policy[kind] = x
    id_policy[kind] = y

get_act_behavior   = lambda obs: int( policy["behavior"]  .predict(obs)[0] )
get_act_evaluation = lambda obs: int( policy["evaluation"].predict(obs)[0] )

get_act = { "behavior": get_act_behavior, "evaluation": get_act_evaluation, }

trying to find id_policy in data/gymnasium/cartpole/policy.json
trying to load policy from data/gymnasium/cartpole/2025-02-06T12:51:54.519095/policy.zip
trying to find id_policy in data/gymnasium/cartpole/policy.json
trying to load policy from data/gymnasium/cartpole/2025-02-06T12:52:22.614772/policy.zip


In [5]:
dataset = {}
id_dataset = {}

for kind in kinds:
    x, y = get_dataset(
        os.path.join(dir_base, id_policy[kind]),
        env_absorbing_time_limit,
        get_act[kind],
        {
            "seed": seed,
            "n_trajectories": n_trajectories,
            "max_episode_steps": max_episode_steps,
        },
        verbosity=1,
    )

    dataset[kind] = x
    id_dataset[kind] = y

trying to find id_dataset in data/gymnasium/cartpole/2025-02-06T12:51:54.519095/dataset.json
trying to load dataset from data/gymnasium/cartpole/2025-02-06T12:51:54.519095/2025-02-06T14:56:13.508204/dataset.parquet
trying to find id_dataset in data/gymnasium/cartpole/2025-02-06T12:52:22.614772/dataset.json
trying to load dataset from data/gymnasium/cartpole/2025-02-06T12:52:22.614772/2025-02-06T14:56:23.202427/dataset.parquet


In [38]:
from plugins.gymnasium.cartpole.dataset import get_dataset_cartpole

In [211]:
dir_policy = {
    kind: os.path.join(dir_base, id_policy[kind])
        for kind in kinds
}
dir_dataset = {
    kind: os.path.join(dir_policy[kind], id_dataset[kind])
        for kind in kinds
}

ds = get_dataset_cartpole(dir_dataset["behavior"], dir_policy["evaluation"])
ds.head()

,obs_init,obs,act,rew,obs_next,id,t,probs_init,probs_next
0,"[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]","[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]",1,1.0,"[-0.04418381, 0.19318062, 0.028309302, -0.29017842]",0,0,"[0.584713, 0.41528696]","[0.91418236, 0.08581759]"
1,"[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]","[-0.04418381, 0.19318062, 0.028309302, -0.29017842]",0,1.0,"[-0.040320195, -0.0023333223, 0.022505734, 0.011296801]",0,1,"[0.584713, 0.41528696]","[0.56865233, 0.43134767]"
2,"[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]","[-0.040320195, -0.0023333223, 0.022505734, 0.011296801]",0,1.0,"[-0.040366862, -0.19777068, 0.022731671, 0.31099474]",0,2,"[0.584713, 0.41528696]","[0.12171043, 0.87828964]"
3,"[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]","[-0.040366862, -0.19777068, 0.022731671, 0.31099474]",1,1.0,"[-0.04432228, -0.0029798457, 0.028951565, 0.025566498]",0,3,"[0.584713, 0.41528696]","[0.46339622, 0.5366037]"
4,"[-0.044153366, -0.001522139, 0.028441364, -0.0066030375]","[-0.04432228, -0.0029798457, 0.028951565, 0.025566498]",1,1.0,"[-0.044381876, 0.19171521, 0.029462896, -0.25784314]",0,4,"[0.584713, 0.41528696]","[0.8670787, 0.13292128]"


In [210]:
import pandas as pd

pd.set_option('display.max_rows', 200)
ds[ds["id"] == 1]

,obs_init,obs,act,rew,obs_next,id,t,probs_init,probs_next
200,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.005118339, 0.012782726, -0.026331443, 0.04020888]",1,1.0,"[0.005373993, 0.20827217, -0.025527265, -0.2606642]",1,0,"[0.7643944, 0.2356057]","[0.9662921, 0.03370792]"
201,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.005373993, 0.20827217, -0.025527265, -0.2606642]",0,1.0,"[0.009539437, 0.013523757, -0.03074055, 0.02385906]",1,1,"[0.7643944, 0.2356057]","[0.8244763, 0.17552373]"
202,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.009539437, 0.013523757, -0.03074055, 0.02385906]",1,1.0,"[0.009809912, 0.20907277, -0.030263368, -0.27836215]",1,2,"[0.7643944, 0.2356057]","[0.9763321, 0.023667887]"
203,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.009809912, 0.20907277, -0.030263368, -0.27836215]",0,1.0,"[0.013991367, 0.014395322, -0.035830613, 0.0046242108]",1,3,"[0.7643944, 0.2356057]","[0.8787256, 0.121274404]"
204,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.013991367, 0.014395322, -0.035830613, 0.0046242108]",0,1.0,"[0.014279273, -0.18019493, -0.03573813, 0.2857903]",1,4,"[0.7643944, 0.2356057]","[0.48952013, 0.5104798]"
205,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.014279273, -0.18019493, -0.03573813, 0.2857903]",1,1.0,"[0.010675374, 0.015417997, -0.030022321, -0.017946508]",1,5,"[0.7643944, 0.2356057]","[0.892086, 0.10791401]"
206,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.010675374, 0.015417997, -0.030022321, -0.017946508]",0,1.0,"[0.010983734, -0.17926082, -0.030381251, 0.2651149]",1,6,"[0.7643944, 0.2356057]","[0.5200229, 0.479977]"
207,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.010983734, -0.17926082, -0.030381251, 0.2651149]",1,1.0,"[0.0073985183, 0.016281286, -0.025078954, -0.03699363]",1,7,"[0.7643944, 0.2356057]","[0.9024426, 0.09755744]"
208,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.0073985183, 0.016281286, -0.025078954, -0.03699363]",0,1.0,"[0.007724144, -0.17847222, -0.025818827, 0.24767232]",1,8,"[0.7643944, 0.2356057]","[0.54619235, 0.45380762]"
209,"[0.005118339, 0.012782726, -0.026331443, 0.04020888]","[0.007724144, -0.17847222, -0.025818827, 0.24767232]",1,1.0,"[0.0041546994, 0.017008768, -0.020865379, -0.05304132]",1,9,"[0.7643944, 0.2356057]","[0.9105602, 0.08943991]"


In [49]:
env = gym.make('CartPole-v0')
specs = get_specs_env(env)

In [50]:
specs

{'obs': {'shape': (4,),
  'dtype': dtype('float32'),
  'min': array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
        dtype=float32),
  'max': array([4.8000002e+00, 3.4028235e+38, 4.1887903e-01, 3.4028235e+38],
        dtype=float32)},
 'act': {'shape': (), 'dtype': dtype('int64'), 'min': 0, 'max': 1}}

In [8]:
obs, _ = env_absorbing_time_limit.reset()

rew_array = []

done = False
t = 0
while not done:
    act = get_act_behavior(obs)
    obs, rew, truncated, terminated, _ = env_absorbing_time_limit.step(act)
    rew_array.append(rew)
    t += 1
    done = truncated or terminated

np.array(rew_array)

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,
       -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1

In [10]:
for kind in kinds:
    print(f"{kind} policy")

    for i in [-1, 1]:
        print(f"{i=}", end=": ")
        a = np.sum(dataset[kind]["rew"] == i)
        b = dataset[kind]["rew"].size
        print(a / b)

    print()

behavior policy
i=-1: 0.26671
i=1: 0.73329

evaluation policy
i=-1: 0.005
i=1: 0.995

